In [1]:
import ee
import geopandas as gpd
from datetime import datetime
import calendar

# Authenticate and initialize GEE
ee.Authenticate()
ee.Initialize()

# Load your area of interest shapefile
shapefile_path = "/datasets/work/ev-veg-height/work/Tasks/Carbon_stock/V2.0/boundary/combined_areas/combined_areas.shp"
gdf = gpd.read_file(shapefile_path)

# Ensure the shapefile is in WGS 84 (EPSG:4326)
gdf = gdf.to_crs(epsg=4326)

# Convert shapefile to GeoJSON and then to Earth Engine FeatureCollection
aoi = ee.FeatureCollection(gdf.__geo_interface__)

# Function to calculate NDVI and keep NIR and Red bands
def calculate_ndvi(image):
    # Select NIR (B8) and Red (B4) bands and cast to Float32
    nir = image.select('B8').toFloat().rename('NIR')
    red = image.select('B4').toFloat().rename('Red')
    
    # Calculate NDVI (already Float32)
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    
    # Return image with NDVI, NIR, and Red bands (all Float32)
    return image.addBands([ndvi, nir, red])

# Loop through years and months
for year in range(2014, 2025):  # From 2020 to 2025
    for month in range(1, 13):  # All 12 months
        # Calculate start and end dates for the month
        start_date = datetime(year, month, 1)
        days_in_month = calendar.monthrange(year, month)[1]
        end_date = datetime(year, month, days_in_month)
        
        # Format dates as strings
        start_date_str = start_date.strftime('%Y-%m-%d')
        end_date_str = end_date.strftime('%Y-%m-%d')
        
        # Load Sentinel-2 image collection
        s2 = ee.ImageCollection('COPERNICUS/S2') \
            .filterDate(start_date_str, end_date_str) \
            .filterBounds(aoi) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        
        # Apply NDVI calculation (also adds NIR and Red bands)
        s2_processed = s2.map(calculate_ndvi)
        
        # Reduce to max NDVI composite (this will also select corresponding NIR and Red values)
        max_composite = s2_processed.qualityMosaic('NDVI')
        
        # Clip to AOI
        max_composite_clipped = max_composite.clip(aoi)
        
        # Select NDVI, NIR, and Red bands for export (stacked in this order)
        stacked_bands = max_composite_clipped.select(['NDVI', 'NIR', 'Red'])
        
        # Export the result
        task = ee.batch.Export.image.toDrive(
            image=stacked_bands,
            description=f'NDVI_NIR_Red_{year}_{month:02d}',
            folder='EarthEngineExports',
            fileNamePrefix=f'ndvi_nir_red_{year}_{month:02d}',
            region=aoi.geometry().bounds().getInfo()['coordinates'],
            scale=10,
            crs='EPSG:4326',
            maxPixels=1e13
        )
        
        # Start the export task
        task.start()
        print(f"Export task for {start_date_str} to {end_date_str} started: NDVI, NIR, Red stacked")

/home/jos066/.local/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


Export task for 2014-01-01 to 2014-01-31 started: NDVI, NIR, Red stacked
Export task for 2014-02-01 to 2014-02-28 started: NDVI, NIR, Red stacked
Export task for 2014-03-01 to 2014-03-31 started: NDVI, NIR, Red stacked
Export task for 2014-04-01 to 2014-04-30 started: NDVI, NIR, Red stacked
Export task for 2014-05-01 to 2014-05-31 started: NDVI, NIR, Red stacked
Export task for 2014-06-01 to 2014-06-30 started: NDVI, NIR, Red stacked
Export task for 2014-07-01 to 2014-07-31 started: NDVI, NIR, Red stacked
Export task for 2014-08-01 to 2014-08-31 started: NDVI, NIR, Red stacked
Export task for 2014-09-01 to 2014-09-30 started: NDVI, NIR, Red stacked
Export task for 2014-10-01 to 2014-10-31 started: NDVI, NIR, Red stacked
Export task for 2014-11-01 to 2014-11-30 started: NDVI, NIR, Red stacked
Export task for 2014-12-01 to 2014-12-31 started: NDVI, NIR, Red stacked
Export task for 2015-01-01 to 2015-01-31 started: NDVI, NIR, Red stacked
Export task for 2015-02-01 to 2015-02-28 started: N